In [16]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, root_mean_squared_error

In [4]:
df = pd.read_csv('/mnt/wines.csv')
df.sample(1)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,type,quality
3512,8.3,0.23,0.3,2.1,0.049,21.0,153.0,0.9953,3.09,0.5,9.6,white,6


*Let's start with one-hot encoding the type column!*

In [7]:
df = pd.get_dummies(df, columns = ['type'], drop_first = True, dtype = 'uint8')
df.sample(1)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,type_white
4088,6.1,0.28,0.24,19.95,0.074,32.0,174.0,0.99922,3.19,0.44,9.3,6,1


In [8]:
df.shape

(6497, 13)

*Running the regression on full dataset:*

In [10]:
model = sm.GLM.from_formula(formula = 'quality ~ type_white + alcohol + sulphates + pH + density + total_sulfur_dioxide + free_sulfur_dioxide + chlorides + residual_sugar + citric_acid + volatile_acidity + fixed_acidity',
                            data = df,
                            family = sm.families.Poisson()) # I'm choosing poisson instead of bionomial here due to our target is no longer binary
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                quality   No. Observations:                 6497
Model:                            GLM   Df Residuals:                     6484
Model Family:                 Poisson   Df Model:                           12
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -12050.
Date:                Sat, 17 Jan 2026   Deviance:                       601.42
Time:                        12:05:25   Pearson chi2:                     596.
No. Iterations:                     4   Pseudo R-squ. (CS):            0.03812
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               18.5668      8.206      2.263      0.024       2.484      34.650
type_white              -0.0622      0.032     -1.914      0.056      -0.126       0.001
alcohol                  0.0376      0.010      3.603      0.000       0.017       0.058
sulphates                0.1237      0.043      2.899      0.004       0.040       0.207
pH                       0.0872      0.051      1.697      0.090      -0.013       0.188
density                -17.6556      8.320     -2.122      0.034     -33.962      -1.349
total_sulfur_dioxide    -0.0002      0.000     -1.329      0.184      -0.001       0.000
free_sulfur_dioxide      0.0008      0.000      1.962      0.050    1.01e-06       0.002
chlorides               -0.1403      0.196     -0.717      0.473      -0.524       0.243
residual_sugar           0.0107      0.003      3.144      0.002       0.004       0.017
citric_acid             -0.0128      0.046     -0.281      0.779      -0.102       0.077
volatile_acidity        -0.2690      0.047     -5.716      0.000      -0.361      -0.177
fixed_acidity            0.0148      0.009      1.640      0.101      -0.003       0.032
========================================================================================
"""

**Notice:**
* type_white: P-value = 0.056
* pH: P-value = 0.09
* total_sulfur_dioxide: P-value = 0.184
* free_sulfur_dioxide: P-value = 0.05
* chlorides: P-value = 0.473
* citric_acid: P-value = 0.779
* fixed_acidity: P-value = 0.101

All of which has a P-value of greater than 0.05 (a common significance level).

We can consider dropping these features to simplify our model and improve computational efficiency.

In [24]:
model_dropped  = sm.GLM.from_formula(formula = 'quality ~ alcohol + sulphates + density + residual_sugar + volatile_acidity',
                                     family = sm.families.Poisson(),
                                     data=df)
dropped_results = model_dropped.fit()
dropped_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                quality   No. Observations:                 6497
Model:                            GLM   Df Residuals:                     6491
Model Family:                 Poisson   Df Model:                            5
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -12057.
Date:                Sat, 17 Jan 2026   Deviance:                       617.13
Time:                        12:27:09   Pearson chi2:                     611.
No. Iterations:                     4   Pseudo R-squ. (CS):            0.03579
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.9806      3.490      0.281      0.779      -5.860       7.821
alcohol              0.0588      0.006      9.241      0.000       0.046       0.071
sulphates            0.1178      0.041      2.899      0.004       0.038       0.197
density              0.1632      3.484      0.047      0.963      -6.665       6.992
residual_sugar       0.0033      0.002      2.048      0.041       0.000       0.006
volatile_acidity    -0.2433      0.038     -6.352      0.000      -0.318      -0.168
====================================================================================
"""

In [25]:
before_r2 = r2_score(df['quality'].values, result.predict())
after_r2 = r2_score(df['quality'].values, dropped_results.predict())
before_rmse = root_mean_squared_error(df['quality'].values, result.predict())
after_rmse = root_mean_squared_error(df['quality'].values, dropped_results.predict())
print(f'Before dropping insignificant features R^2 score:\t{before_r2:.2f}\nAfter dropping insignificant features:\t{after_r2:.2f}')
print('\n')
print(f'Before dropping insignificant features RMSE score:\t{before_rmse:.2f}\nAfter dropping insignificant features:\t{after_rmse:.2f}')

Before dropping insignificant features R^2 score:	0.30
After dropping insignificant features:	0.28


Before dropping insignificant features RMSE score:	0.73
After dropping insignificant features:	0.74


*Relatively low!*

**Let's try stratification**

In [35]:
from sklearn.model_selection import train_test_split
scores = []
for i in range(100):
  X_train, X_test, y_train, y_test = train_test_split(df, df.quality, stratify = df.quality, test_size = 0.3, random_state = None)
  mod  = sm.GLM.from_formula(formula = 'quality ~ type_white + alcohol + sulphates + pH + density + total_sulfur_dioxide + free_sulfur_dioxide + chlorides + residual_sugar + citric_acid + volatile_acidity + fixed_acidity',
                               family = sm.families.Poisson(),
                               data=X_train)
  res = mod.fit()
  preds = res.predict(X_test)
  scores.append(r2_score(y_test, preds))
print(f'Stratified R-squared: {scores[-1]:.4f}')

Stratified R-squared: 0.2920


*The figures did improve!*

Let's try running 10-fold Cross Validation

In [41]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 10, shuffle = True, random_state = None)

for train, test in kf.split(df.index.values):
  model = sm.GLM.from_formula(formula = 'quality ~ type_white + alcohol + sulphates + pH + density + total_sulfur_dioxide + free_sulfur_dioxide + chlorides + residual_sugar + citric_acid + volatile_acidity + fixed_acidity',
                            data = df.iloc[train],
                            family = sm.families.Poisson())
  result = model.fit()
  predsTrain = result.predict(df.iloc[train])
  preds = result.predict(df.iloc[test])
  print(f'Train R2: {r2_score(df.quality.iloc[train], predsTrain):.4f}')
  print(f'Test R2: {r2_score(df.quality.iloc[test], preds):.4f}')

Train R2: 0.2974
Test R2: 0.2883
Train R2: 0.2983
Test R2: 0.2824
Train R2: 0.2921
Test R2: 0.3386
Train R2: 0.3006
Test R2: 0.2546
Train R2: 0.2956
Test R2: 0.3070
Train R2: 0.2974
Test R2: 0.2886
Train R2: 0.2948
Test R2: 0.3108
Train R2: 0.2977
Test R2: 0.2852
Train R2: 0.2938
Test R2: 0.3200
Train R2: 0.3015
Test R2: 0.2545


Training accuracy seems to be higher than test accuracy. Let's test overfitting possibility for this model

In [49]:
scores = []
for i in range(100):
  X_train, X_test, y_train, y_test = train_test_split(df, df.quality, stratify = df.quality, test_size = 0.9, random_state = None)
  mod  = sm.GLM.from_formula(formula = 'quality ~ type_white + alcohol + sulphates + pH + density + total_sulfur_dioxide + free_sulfur_dioxide + chlorides + residual_sugar + citric_acid + volatile_acidity + fixed_acidity',
                               family = sm.families.Poisson(),
                               data=X_train)
  res = mod.fit()
  preds = res.predict(X_test)
  scores.append(r2_score(y_test, preds))
print(f'R-squared with only 10% data to train with: {scores[-1]:.4f}')

R-squared with only 10% data to train with: 0.2758


Differences are relatively low, which shows that the model is resilient to overfitting.


Let's see how the results vary with 10x of 10-fold CV

In [51]:
for i in range(10):
  trainRes = []
  valRes = []
  kf = KFold(n_splits=10, shuffle = True, random_state= None)
  for train, test in kf.split(df.index.values):
    model = sm.GLM.from_formula(formula = 'quality ~ type_white + alcohol + sulphates + pH + density + total_sulfur_dioxide + free_sulfur_dioxide + chlorides + residual_sugar + citric_acid + volatile_acidity + fixed_acidity',
                              data = df.iloc[train],
                              family = sm.families.Poisson())
    result = model.fit()
    predsTrain = result.predict(df.iloc[train])
    preds = result.predict(df.iloc[test])

    trainRes.append(r2_score(df.iloc[train].quality, predsTrain))
    valRes.append(r2_score(df.iloc[test].quality, preds))
  print(f'Train R2: {round(np.mean(trainRes), 4)}\nTest R2: {round(np.mean(valRes), 4)}')


Train R2: 0.2969
Test R2: 0.2911
Train R2: 0.297
Test R2: 0.2892
Train R2: 0.297
Test R2: 0.2929
Train R2: 0.2969
Test R2: 0.2922
Train R2: 0.2969
Test R2: 0.2938
Train R2: 0.2969
Test R2: 0.2917
Train R2: 0.297
Test R2: 0.291
Train R2: 0.2969
Test R2: 0.2926
Train R2: 0.2969
Test R2: 0.2938
Train R2: 0.297
Test R2: 0.2908


The results remained stable, at the same time the problem of overfitting is relatively low due to the difference in R2-squared for training set and validation set is not too great apart.